# Create dataset of expressions for training similar-expressions model

## Test nesymres data generation

In [4]:
!export PYTHONPATH="/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/src/dataset_generation/nesymres:$PYTHONPATH"

In [5]:
!python nesymres/data_creation/dataset_creation.py --number_of_equations 1000 --no-debug

Traceback (most recent call last):
  File "/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/src/dataset_generation/nesymres/data_creation/dataset_creation.py", line 19, in <module>
    from utils import create_env, H5FilesCreator
  File "/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/src/dataset_generation/nesymres/utils.py", line 6, in <module>
    from .dataset import generator
ImportError: attempted relative import with no known parent package


In [59]:
import pickle
import numpy as np
import h5py
import os
from io import BytesIO
from nesymres import dclasses, utils

class CustomUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'dclasses':
            return getattr(dclasses, name)
        elif module == 'utils':
            return getattr(utils, name)
        return super().find_class(module, name)

def load_eq(path_folder, idx, num_eqs_per_set):
    index_file = str(int(idx/num_eqs_per_set))
    f = h5py.File(os.path.join(path_folder, f"{index_file}.h5"), 'r')
    dataset_metadata = f[str(idx - int(index_file)*int(num_eqs_per_set))]
    raw_metadata = np.array(dataset_metadata)
    
    # Create a BytesIO object from the raw_metadata bytes
    metadata_bytes = BytesIO(raw_metadata.tobytes())
    
    # Use the CustomUnpickler with the BytesIO object
    metadata = CustomUnpickler(metadata_bytes).load()
    
    f.close()
    return metadata


fpath = '/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/src/dataset_generation/nesymres/data_creation/data/raw_datasets/debug/200'
a = load_eq(fpath, 0, 100)

In [62]:
pprint(a)

Equation(code=<code object _lambdifygenerated at 0x139aa31b0, file "<lambdifygenerated-1>", line 1>,
         expr='(({cm_0})*(({ca_0})+(x_1)))+((-1)*(({cm_1})*((({ca_1})+(x_3))*((({cm_2})*(({ca_2})+(x_3)))+(exp(x_2))))))',
         coeff_dict={'ca_0': 'ca_0',
                     'ca_1': 'ca_1',
                     'ca_2': 'ca_2',
                     'cm_0': 'cm_0',
                     'cm_1': 'cm_1',
                     'cm_2': 'cm_2'},
         variables={'x_3', 'x_1', 'x_2'},
         support=None,
         tokenized=None,
         valid=True,
         number_of_points=None)


In [93]:
from typing import Tuple
from nesymres.dclasses import Equation
from torch.distributions.uniform import Uniform
import random


def sample_symbolic_constants(eq: Equation, cfg=None) -> Tuple:
    """Given an equation, returns randomly sampled constants and dummy contants

    Args:
      eq: an Equation.
      cfg: Used for specifying how many and in which range to sample constants. If None, consts equal to dummy_consts

    Returns:
      consts: 
      dummy_consts: 
    """

    dummy_consts = {const: 1 if const[:2] == "cm" else 0 for const in eq.coeff_dict.keys()}
    consts = dummy_consts.copy()
    if cfg:
        used_consts = random.randint(0, min(len(eq.coeff_dict),cfg.num_constants))
        symbols_used = random.sample(list(set(eq.coeff_dict.keys())), used_consts)
        print(f'{used_consts = }', f'{symbols_used = }')
        for si in symbols_used:
            if si[:2] == "ca":
                consts[si] = round(float(Uniform(cfg.additive.min, cfg.additive.max).sample()),3)
            elif si[:2] == "cm":
                consts[si] = round(float(Uniform(cfg.multiplicative.min, cfg.multiplicative.max).sample()),3)
            else:
                raise KeyError
    else:
        consts = dummy_consts
    return consts, dummy_consts

from types import SimpleNamespace

cfg = SimpleNamespace(
    num_constants=10,
    additive=SimpleNamespace(min=0, max=10),
    multiplicative=SimpleNamespace(min=0, max=10)
)

consts, dummy_consts = sample_symbolic_constants(a, cfg)
consts

used_consts = 4 symbols_used = ['ca_2', 'ca_1', 'ca_0', 'cm_2']


{'cm_0': 1,
 'cm_1': 1,
 'cm_2': 7.92,
 'ca_0': 5.028,
 'ca_1': 5.646,
 'ca_2': 1.669}

({'cm_0': 1, 'cm_1': 1, 'cm_2': 1, 'ca_0': 0, 'ca_1': 0, 'ca_2': 0},
 {'cm_0': 1, 'cm_1': 1, 'cm_2': 1, 'ca_0': 0, 'ca_1': 0, 'ca_2': 0})

In [13]:
import sympy as sp

sp.sympify(a.expr)

TypeError: unsupported operand type(s) for +: 'set' and 'Symbol'

## Using DLSM generator directly

In [55]:
from importlib import reload
from nesymres.dataset import generator
reload(generator)
from nesymres.utils import create_env
from pprint import pprint
import sympy as sp
from itertools import chain
import numpy as np

# np.random.seed(2)

env, param, config_dict = create_env("./nesymres/dataset_configuration.json")
pprint(param)

prefix, variables = env.generate_equation(np.random)
print(prefix)

GeneratorDetails(max_len=10,
                 operators='add:10,mul:10,sin:4',
                 max_ops=3,
                 rewrite_functions='',
                 variables=['x_1'],
                 eos_index=1,
                 pad_index=0)
New stack: ['sin', None]
New stack: ['sin', 'add', None, None]
New stack: ['sin', 'add', None, 'add', None, None]
leaves = [['x_1'], ['x_1'], ['x_1']]
Finally: stack = ['sin', 'add', 'x_1', 'add', 'x_1', 'x_1']
f_expr = ['sin', 'add', 'x_1', 'add', 'x_1', 'x_1']
infix = '(sin((x_1)+((x_1)+(x_1))))'
Process equation
3, f = sin(3*x_1)
4, f = cm*sin(3*cm**2*x_1)
5, f = cm*(ca + sin(cm**(ca + 2)*(ca + 3)*(ca + x_1)))
Process equation done
f = cm*(ca + sin(cm**(ca + 2)*(ca + 3)*(ca + x_1)))
f_prefix = ['mul', 'cm', 'add', 'ca', 'sin', 'mul', 'pow', 'cm', 'add', '2', 'ca', 'mul', 'add', '3', 'ca', 'add', 'ca', 'x_1']
['mul', 'cm', 'add', 'ca', 'sin', 'mul', 'pow', 'cm', 'add', '2', 'ca', 'mul', 'add', '3', 'ca', 'add', 'ca', 'x_1']


## Using custom generator

In [149]:
from importlib import reload
from custom import utils, generator, dclasses
reload(generator)
reload(utils)
reload(dclasses)
from custom.utils import create_generator

import numpy as np

# np.random.seed(2)

gen, params = create_generator('./custom/data_settings.json')

expressions = [gen.generate_equation(np.random, -1e3, 1e3, simplify=False, check_const_range=True) for _ in range(10000)]


datapath = '/Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/data'

import os

# Filter out empty strings
expressions = [expr for expr in expressions if expr]

# Create the full path for the output file
output_file = os.path.join(datapath, 'expr_240807_1.txt')

# Write expressions to the file
with open(output_file, 'w') as f:
    for expr in expressions:
        f.write(f"{expr}\n")

print(f"Expressions have been written to {output_file}")



Expressions have been written to /Users/luis/Desktop/Cranmer 2024/Workplace/smallMutations/similar-expressions/data/expr_240807_1.txt


In [64]:
f = gen.replace_const_placeholder(f, 1e16, 1e17)
f

['add', 'add', 8.140416442852966e+16, 'x_1', 'sin', 'x_1']

In [77]:
bool(None)

False

In [72]:
isinstance(f[2], float)

True

In [69]:
list(map(type, f))

[numpy.str_, numpy.str_, float, str, numpy.str_, str]

In [97]:
f = gen._generate_expr(nb_total_ops=3, rng=np.random)
infix = gen.prefix_to_infix(f, coefficients=gen.coefficients, variables=gen.variables)
infix

'((sin(sin(c)))*(x_1))'

In [98]:
expr = gen.infix_to_sympy(infix, gen.variables, gen.rewrite_functions)
expr

x_1*sin(sin(c))

In [99]:
f_prefix = gen.sympy_to_prefix(expr)
f_prefix

['mul', 'x_1', 'sin', 'sin', 'c']